<h1 align = "center"> TWITTER ANALYTICS APPLICATION </h1>

## LIBRARIES

In [1]:
#load libraries
import tweepy
from tweepy import OAuthHandler
import json
import wget
import csv
import numpy as np
import pandas as pd
import re
import webbrowser

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer 
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost, string

import tensorflow
import keras
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

from datetime import datetime
from plotly import graph_objs as go

import plotly.tools as plotly_tools

from plotly.offline import init_notebook_mode,plot
from plotly import tools

import chart_studio 
chart_studio.tools.set_credentials_file(username='musarath', api_key='hN1B3D9TpmpUSxRG5iKy')
import chart_studio.plotly as py
import plotly.graph_objs as go

# included flask
from flask import Flask, request, render_template, jsonify
import import_ipynb

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



Using TensorFlow backend.


## DATA DOWNLOADING

In [2]:
# get tweets from twitter 
def twitter_credentials(hashtag):
    # Twitter API credentials
    
    consumer_key = 'J5CMEPQjJLnm2Wo3IQs8V2qzr'
    consumer_secret ='nUGDFg461iszY7IXfiwVRE9dE9WfdDDNh6TG3Wrw82nYhyRi55'
    access_key = '203862650-MKbLxvUyNurxQDjxhiIndLbr3fH2ZX2GazuOyrFl'
    access_secret =  'NMI6hhi8rmJ6jIhQbhNgsjAB7pwrYuIzcg4tHB5J4unHz'

    # Create the api endpoint
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True)


    twts = []
    print("Downloading in progress...")
    for tweet in tweepy.Cursor( api.search, q='#' + hashtag  + "-filter:retweets",
                                #api.search, q='#' + hashtag + since:2019-09-01 + until:2019-9-22 +  "-filter:retweets" ,
                               lang="en").items(10000):
        twts.append([str(tweet.created_at),tweet.text.encode('utf-8')])
    print("Downloading completed!")

     
    return twts
 

In [3]:
# write to csv file
def write_tweets_to_file(hash,tweets):
    with open('tweets_with_hashtag_'+hash+'.csv', 'w',encoding='utf8',newline='') as the_file:
        writer = csv.writer(the_file)
        writer.writerow(['created_at', 'text'])
        writer.writerows(tweets)
        

In [4]:
# cross check the tweets saved
def read_tweets_from_file(hash):
    tweets = []
    with open('tweets_with_hashtag_'+hash+'.csv', 'r') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            if "fruit" not in row:
                tweets.append(row)
    
    return tweets


## DATA PREPROCESSING

In [5]:
# function to remove http link in tweet text
def remove_http(tweets):
    http_removed_tweets = []
    for idx,row in enumerate(tweets):
        http_removed_tweets.append(re.sub('[n]?http[s]?://\S+', '', row))
    return http_removed_tweets   


In [6]:
# Tweets text cleanup
#to remove the pattern ‘@user’ from all the tweets in our data.

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

def tweets_clean_up(get_tweets):

    get_tweets = pd.DataFrame(get_tweets)
    get_tweets.rename(columns=get_tweets.iloc[0],inplace = True)
    get_tweets.drop(get_tweets.index[0],inplace = True)
    #print(get_tweets.head())

    # remove http link
    get_tweets['tidy_tweet'] = remove_http(get_tweets['text'])

    # remove twitter handles (@user)
    get_tweets['tidy_tweet'] = np.vectorize(remove_pattern)(get_tweets['tidy_tweet'], "@[\w]*")
    
    # remove special characters, numbers, punctuations
    get_tweets['tidy_tweet'] = get_tweets['tidy_tweet'].str.replace("[^a-zA-Z]", " ")

        
    # removing the short words
    get_tweets['tidy_tweet'] = get_tweets['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
        
    return get_tweets
    

In [7]:
# tokenization method
def tweet_tokenization(tweet):
    # Tokenization
    tokenized_tweet = tweet['tidy_tweet'].apply(lambda x: x.split())    
    return tokenized_tweet

# # tokenize the tweets
#tweets_data['tokens'] =tweet_tokenization(tweets_data)
#tweets_data['tokens'].head()

In [8]:
# stemming / Lemmatizier
def tweet_reduce_wordforms(tokenized_tweet):
    #Stemming
    #tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
    #tokenized_tweet.head()
    
    lmtzr = WordNetLemmatizer()
    tokenized_tweet = tokenized_tweet.apply(lambda x: [lmtzr.lemmatize(i,'v') for i in x])
    
    for i in range(1,len(tokenized_tweet)+1):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
    return tokenized_tweet

# # get tokens to baseform
#tweets_data['tokens'] = tweet_reduce_wordforms(tweets_data['tokens'])


## MODELLING

In [9]:
model_train_data = pd.read_csv("tweets_with_cleaned_hashtag_samsung_classification.csv")
train_DF = pd.DataFrame()
train_DF['tweet_text'] = model_train_data['tidy_tweet']
train_DF['label'] = model_train_data['Abstract']

In [10]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_DF['tweet_text'], train_DF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [11]:
# word level tf-idf
tfidf_word = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_word.fit(train_DF['tweet_text'])

xtrain_tfidf_word =  tfidf_word.transform(train_x)
xvalid_tfidf_word =  tfidf_word.transform(valid_x)

# ngram level tf-idf 
tfidf_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_ngram.fit(train_DF['tweet_text'])
xtrain_tfidf_ngram =  tfidf_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_ngram.transform(valid_x)

# characters level tf-idf
tfidf_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_chars.fit(train_DF['tweet_text'])
xtrain_tfidf_chars =  tfidf_chars.transform(train_x) 
xvalid_tfidf_chars =  tfidf_chars.transform(valid_x)

In [12]:
def train_classifier_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.classification_report(predictions, valid_y)
#accuracy_score(predictions, valid_y)

In [13]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("Naive Bayes, WordLevel TF-IDF:\n" , accuracy)
#print ("Naive Bayes, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Naive Bayes, N-Gram Vectors:\n" , accuracy)
#print( "Navie Bayes, N-Gram Vectors: {}%".format(round(accuracy*100,2)))

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_classifier_model(naive_bayes.MultinomialNB(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
print ("Naive Bayes, CharLevel TF-IDF:\n" , accuracy)
#print ("Navie Bayes, CharLevel Vectors: {}%".format(round(accuracy*100,2)))

Naive Bayes, WordLevel TF-IDF:
               precision    recall  f1-score   support

           0       0.67      1.00      0.81        31
           1       0.74      0.84      0.79        55
           2       0.98      0.61      0.75       138
           3       0.42      1.00      0.59        15
           4       0.00      0.00      0.00         0

    accuracy                           0.74       239
   macro avg       0.56      0.69      0.59       239
weighted avg       0.85      0.74      0.76       239

Naive Bayes, N-Gram Vectors:
               precision    recall  f1-score   support

           0       0.48      1.00      0.65        22
           1       0.45      0.65      0.53        43
           2       0.90      0.46      0.61       168
           3       0.17      1.00      0.29         6
           4       0.00      0.00      0.00         0

    accuracy                           0.56       239
   macro avg       0.40      0.62      0.41       239
weighted avg   

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [14]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print( "Logistic Regression, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "Logistic Regression, WordLevel TF-IDF:\n",accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("Logistic Regression, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("Logistic Regression, N-Gram Vectors: \n",accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_classifier_model(linear_model.LogisticRegression(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
#print( "Logistic Regression, CharLevel Vectors:{}%".format(round(accuracy*100,2)))
print( "Logistic Regression, CharLevel Vectors:\n",accuracy)

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning:

Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.



Logistic Regression, WordLevel TF-IDF:
               precision    recall  f1-score   support

           0       0.63      1.00      0.77        29
           1       0.90      0.79      0.84        71
           2       0.95      0.69      0.80       118
           3       0.58      1.00      0.74        21
           4       0.00      0.00      0.00         0

    accuracy                           0.79       239
   macro avg       0.61      0.70      0.63       239
weighted avg       0.87      0.79      0.81       239

Logistic Regression, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.37      1.00      0.54        17
           1       0.39      0.75      0.51        32
           2       0.94      0.44      0.60       186
           3       0.11      1.00      0.20         4
           4       0.00      0.00      0.00         0

    accuracy                           0.53       239
   macro avg       0.36      0.64      0.37       23

In [15]:
# SVM on Word Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print( "SVM, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "SVM, WordLevel TF-IDF:/n%",accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("SVM, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("SVM, N-Gram Vectors:\n%",accuracy)


# SVM on Character Level TF IDF Vectors
accuracy = train_classifier_model(svm.SVC(), xtrain_tfidf_chars, train_y, xvalid_tfidf_chars)
#print( "SVM, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "SVM, CharLevel Vectors: \n",accuracy)

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.



SVM, WordLevel TF-IDF:/n%               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.36      0.53       239
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.36       239
   macro avg       0.20      0.07      0.11       239
weighted avg       1.00      0.36      0.53       239

SVM, N-Gram Vectors:
%               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.36      0.53       239
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.36       239
   macro avg       0.20      0.07      0.11       239
weighted avg       1.00     

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning:

The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.



SVM, CharLevel Vectors: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       1.00      0.36      0.53       239
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

    accuracy                           0.36       239
   macro avg       0.20      0.07      0.11       239
weighted avg       1.00      0.36      0.53       239



C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



In [16]:
# RF on Word Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
#print ("Random Forest, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print ("Random Forest, WordLevel TF-IDF: \n%",accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
#print ("Random Forest, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print ("Random Forest, N-Gram Vectors: \n",accuracy)


# SVM on Character Level TF IDF Vectors
accuracy = train_classifier_model(ensemble.RandomForestClassifier(), xtrain_tfidf_chars,train_y,xvalid_tfidf_chars)
#print( "Random Forest, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "Random Forest, CharLevel Vectors: \n",accuracy)

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Random Forest, WordLevel TF-IDF: 
%               precision    recall  f1-score   support

           0       0.70      0.97      0.81        33
           1       0.97      0.63      0.76        95
           2       0.86      0.88      0.87        84
           3       0.67      0.92      0.77        26
           4       0.11      1.00      0.20         1

    accuracy                           0.80       239
   macro avg       0.66      0.88      0.68       239
weighted avg       0.86      0.80      0.81       239

Random Forest, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.43      1.00      0.61        20
           1       0.89      0.36      0.52       151
           2       0.52      0.73      0.61        62
           3       0.14      1.00      0.24         5
           4       0.00      0.00      0.00         1

    accuracy                           0.52       239
   macro avg       0.40      0.62      0.39       239
weighted

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.

C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [17]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, xvalid_tfidf_word.tocsc())
#print( "Extereme Gradient Boosting, WordLevel TF-IDF: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, WordLevel TF-IDF: \n",accuracy)

# Extereme Gradient Boosting on Ngram Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc())
#print( "Extereme Gradient Boosting, N-Gram Vectors: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, N-Gram Vectors: \n",accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_classifier_model(xgboost.XGBClassifier(), xtrain_tfidf_chars.tocsc(), train_y, xvalid_tfidf_chars.tocsc())
#print( "Extereme Gradient Boosting, CharLevel Vectors: {}%".format(round(accuracy*100,2)))
print( "Extereme Gradient Boosting, CharLevel Vectors: \n",accuracy)

Extereme Gradient Boosting, WordLevel TF-IDF: 
               precision    recall  f1-score   support

           0       0.78      0.90      0.84        40
           1       1.00      0.67      0.80        93
           2       0.88      1.00      0.94        76
           3       0.78      1.00      0.88        28
           4       0.22      1.00      0.36         2

    accuracy                           0.85       239
   macro avg       0.73      0.91      0.76       239
weighted avg       0.89      0.85      0.86       239



C:\Users\Musarath\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.



Extereme Gradient Boosting, N-Gram Vectors: 
               precision    recall  f1-score   support

           0       0.43      1.00      0.61        20
           1       0.34      0.70      0.46        30
           2       0.90      0.44      0.59       174
           3       0.28      0.67      0.39        15
           4       0.00      0.00      0.00         0

    accuracy                           0.54       239
   macro avg       0.39      0.56      0.41       239
weighted avg       0.75      0.54      0.56       239

Extereme Gradient Boosting, CharLevel Vectors: 
               precision    recall  f1-score   support

           0       0.89      0.95      0.92        43
           1       0.94      0.89      0.91        65
           2       0.98      0.98      0.98        86
           3       0.94      0.81      0.87        42
           4       0.33      1.00      0.50         3

    accuracy                           0.92       239
   macro avg       0.82      0.93   

## CLASSIFIER MODEL

In [18]:
def train_classifier_final_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
        
    return predictions


In [19]:
def read_new_tweets(file):
    tweets = []
    with open(file, 'r') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:
            tweets.append(row)
    
    return tweets


In [20]:
twitter_tweets = []
# download data
def twitter_data_download(hash_tag):    
    
    # tweets download
    twts_download = twitter_credentials(hash_tag)

    # call the function        
    write_tweets_to_file(hash_tag,twts_download)

    # call read from file function
    twitter_tweets = read_tweets_from_file(hash_tag)
    #print(twitter_tweets)
    
    # tweets cleanup function
    twitter_tweets = tweets_clean_up(twitter_tweets)
    #print(twitter_tweets)
    
    return twitter_tweets



In [21]:
df_tweets_data = []
# function to tag tweets
def twitter_abstract(tweets_data,xtrain_tfidf_word,train_y):
    #tokenize
    tokens_tfidf_word =  tfidf_word.transform(tweets_data['tidy_tweet'])

    pred = train_classifier_final_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, tokens_tfidf_word.tocsc())
    tweets_data['Abstract_class'] = pred
    tweets_data['Abstract'] = encoder.inverse_transform(pred)
    
    #convert the date field to date format 
    df_tweets_data = pd.DataFrame(tweets_data) 

    #convert to date type
    df_tweets_data['created_at'] = pd.to_datetime(df_tweets_data["created_at"])
    df_tweets_data['created_at'] = df_tweets_data['created_at'].dt.date 
    # drop the columns
    df_tweets_data.drop(['text','Abstract_class'],axis = 1,inplace = True)
    return df_tweets_data


## SENTIMENT ANALYSIS

In [22]:
# method to calculate sentiment of each tweet
def tweets_sentiment(df_tweet):
    # next, we initialize VADER so we can use it within our Python script
    sid = SentimentIntensityAnalyzer()
    sentiment_score = []

    ## Calling the polarity_scores method on sid and passing in the title 
    #outputs a dictionary with negative, neutral, positive, and compound scores for the input title
    for tweet in df_tweet['tidy_tweet']:
        scores = sid.polarity_scores(tweet)
        sentiment_score.append(scores['compound'])

    df_tweet['sentiment'] = sentiment_score
    return df_tweet


In [23]:
# function to define polarity
def tweets_polarity(df_tweets):
    polarity = []
    for t in df_tweets['sentiment']:
        if t<0:
            polarity.append("Negative")
        elif t == 0:
            polarity.append("Neutral")
        else:
            polarity.append("Positive")
    df_tweets['Polarity'] = polarity
    return df_tweets

## VISUALIZATIONS

In [24]:
# function for number of tweets per abstract over time
def tweets_over_time(df_tweets):
    df_daily_count = df_tweets.groupby(['created_at','Abstract'],as_index = False).agg({'tidy_tweet': "count"})

    tmp = df_daily_count[df_daily_count['Abstract'] == 'CustomerService']
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=tmp['created_at'], y=tmp['tidy_tweet'],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5,
                  color='rgb(131, 90, 241)'),
        name = 'CustomerService',
        stackgroup='one' # define stack group
    ))
    tmp1 = df_daily_count[df_daily_count['Abstract'] == 'News']
    fig.add_trace(go.Scatter(
        x=tmp1['created_at'], y=tmp1['tidy_tweet'],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5,
            color='rgb(127, 166, 238)'), 
        name = 'News',
        stackgroup='one'
    ))
    tmp2 = df_daily_count[df_daily_count['Abstract'] == 'Price']
    fig.add_trace(go.Scatter(
        x=tmp2['created_at'], y=tmp2['tidy_tweet'],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5,
            color='rgb(111, 231, 219)'), 
        name = 'Price',
        stackgroup='one'
     ))
    tmp3 = df_daily_count[df_daily_count['Abstract'] == 'Products']
    fig.add_trace(go.Scatter(
        x=tmp3['created_at'], y=tmp3['tidy_tweet'],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, 
            color='rgb(184, 247, 212)'),
        name = 'Products',
        stackgroup='one'
     ))
    tmp4 = df_daily_count[df_daily_count['Abstract'] == 'Recommendation']
    fig.add_trace(go.Scatter(
        x=tmp4['created_at'], y=tmp4['tidy_tweet'],
        hoverinfo='x+y',
        mode='lines',
        line=dict(width=0.5, 
            color='rgb(95,158,160)'),
        name = 'Recommendation',
        stackgroup='one'
     ))
    fig.layout.update(title_text='Total activity over time',
                      paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                     margin=dict(l=50, r=10, t=80, b=80))
    fig.update_layout(legend_orientation="h")

    
    plot_url = py.plot(fig, filename='Tweets over time', auto_open=False,)
    #first_plot_url = fig.write_html('first_figure.html', auto_open=True)
    #print(plot_url)
    #fig.show()
    return plot_url
    #return fig


In [25]:
# method to plot sentiments over all categories
def sentiment_analysis_plot(df_tweets,hashtag):
    #plot sentiments w.r.t abstract
    number_of_tweets = str(len(df_tweets['tidy_tweet']))
    df_sentiment_polarity = df_tweets.groupby(['Abstract','Polarity'],as_index = False).agg({'tidy_tweet': "count"})

    tmp = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Positive']
    fig = go.Figure()
    fig.add_trace(go.Bar(    
        y=tmp['Abstract'],
        x=tmp['tidy_tweet'] ,
        name='Positive',
        orientation='h',
        marker=dict(
            color='green',
        )
    ))
    tmp1 = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Neutral']
    fig.add_trace(go.Bar(    
        y=tmp1['Abstract'],
        x=tmp1['tidy_tweet'],
        name='Neutral',
        orientation='h',
        marker=dict(
            color='gold',
        )
    ))
    tmp2 = df_sentiment_polarity[df_sentiment_polarity['Polarity'] == 'Negative']
    fig.add_trace(go.Bar(
        y=tmp2['Abstract'],
        x=tmp2['tidy_tweet'],
        name='Negative',
        orientation='h',
        marker=dict(
            color='red',
        )
    ))
    fig.layout.update(barmode='stack',
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        #paper_bgcolor='rgb(248, 248, 255)',
        #plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=100, r=10, t=80, b=80),
    )
    fig.layout.update(title_text='Data gathered from the analysis of '+ number_of_tweets +' twitter reviews on '+ hashtag.upper())
    fig.update_layout(legend_orientation="h")

    #fig.show()
    plot_url = py.plot(fig, filename='Tweets Sentiment', auto_open=False,)
    
    return plot_url
    #return fig

In [26]:
def sentiment_categorize(df_tweets):
    # categorize sentiment into different categories
    tweets_sent_cat = []
    #df_tweets_1 = {'sentiment': [0.35,0.4]}
    #df_tweets_1 = pd.DataFrame(df_tweets_1)
    for t in df_tweets['sentiment']:
        if t == 0:
            tweets_sent_cat.append("fair") 
        elif t <= -0.9:
            tweets_sent_cat.append("Unpleasant")
        elif t > -0.9 and t <= -0.7:
             tweets_sent_cat.append("disgust")
        elif t > -0.7 and t <= -0.4:
             tweets_sent_cat.append("upset")
        elif t < 0 and t > -0.4:
             tweets_sent_cat.append("sadness")
        elif t >= 0.9:
            tweets_sent_cat.append("pleasant")
        elif t >= 0.7 and t < 0.9:
             tweets_sent_cat.append("elate")
        elif t >= 0.4 and t < 0.7:
             tweets_sent_cat.append("excited")
        elif t >0 and t < 0.4:
             tweets_sent_cat.append("happy")

    df_tweets['tweets_sent_cat'] =  tweets_sent_cat
    
    return df_tweets


In [27]:
import plotly.express as px
# method to plot spiral plot of sentiments
def tweets_spiral(df_tweets):
    df_tweets_sent_cat = df_tweets.groupby(['Abstract','tweets_sent_cat'],as_index = False).agg({'tidy_tweet': "count"})
    #df_tweets_sent_cat

    #plot the chart
    fig = px.bar_polar(df_tweets_sent_cat, r="tidy_tweet", theta="tweets_sent_cat",
                        color="Abstract",template="seaborn",
                       color_discrete_sequence= px.colors.sequential.Plasma[-2::-1])
    fig.layout.update(title_text='Sentiments spiral')
    fig.update_layout(legend_orientation="h",
                      margin=dict(l=80, r=80, t=80, b=100),
                     paper_bgcolor='rgba(0,0,0,0)')
    
    #fig.show()
    plot_url = py.plot(fig, filename='Tweets Spiral Sentiment', auto_open=False,)
    
    return plot_url

In [28]:
# method to display all tweets for the given hashtag in a table
def tweets_table(df_tweets,hashtag):
    df_tweets = df_tweets.loc[1:3000]
    values = [list(df_tweets['created_at']), #1st col
      list(df_tweets['tidy_tweet']),
      list(df_tweets['Abstract'])]


    fig = go.Figure(data=[go.Table(
      columnorder = [1,2,3],
      columnwidth = [80,400,80],
      header = dict(
        values = [['<b>CREATED_AT</b>'],
                    ['<b>TWEETS</b>'],
                 ['<b>ABSTRACT</b>']],
        line_color='darkslategray',
        fill_color='royalblue',
        align=['left','center'],
        font=dict(color='white', size=12),
        height=40
      ),
      cells=dict(
        values=values,
        line_color='darkslategray',
        fill=dict(color=['white','white', 'paleturquoise']),
        align=['left', 'center'],
        font_size=12,
        height=30)
        )
    ])
    fig.layout.update(title_text='Tweets on #'+hashtag.upper(),
                      paper_bgcolor='rgba(0,0,0,0)',
                     font=dict(
                             family="Courier New, monospace",
                             size=18,
                             color="#7f7f7f"
            ))

    
    #fig.show()
    plot_url = py.plot(fig, filename='Tweets Table', auto_open=False,weight = "bold")
    
    return plot_url
    #return fig

## REPORT GENERATION

In [29]:
def define_html_string(hashtag,p1,p2,t1,p3):
    print(hashtag)
    html_string = '''
    <html>
        <head>
            <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css">
            <style>body{ margin:0 100; background:whitesmoke; }</style>
        </head>
        <body>
            <h1><img src = "https://1000logos.net/wp-content/uploads/2017/06/Twitter-Logo.png" alt = "Twitter" 
            width="100" height="100">Twitter Review Analysis on #'''+ hashtag.upper() + ''' </h1> 
            <!-- Add icon library -->
        
            
            <!-- *** Section 1 *** ---> 
            <!--<h2>Tweets over time</h2>-->
            <iframe width="700" height="400" frameborder="0" seamless="seamless" scrolling="no" \
    src="''' + p1 + '''"></iframe>
             <!-- *** Section 2 *** --->
            <!--<h2>Tweets Sentiment Analysis</h2>-->
            <iframe width= "600" height="400" frameborder="0" seamless="seamless" scrolling="no" \
    src="''' + p2 + '''"></iframe> 
            <!--<h2>Twitter Reviews</h2>-->
            <iframe width="800" height="500" frameborder="0" seamless="seamless" scrolling="no" \
    src="''' + t1 + '''"></iframe> 
            <!--<h2>Sentiment Spiral</h2>-->
            <iframe width="500" height="500" frameborder="0" seamless="seamless" scrolling="no" \
    src="''' + p3 + '''"></iframe> 

        </body>
    </html>'''
    
    return html_string

In [30]:
# method to create a report html
def write_to_html(hashtag,html_string):
    f = open('C:/Users/Musarath/Musarath/Anly699_Project/Anly699_TwitterAnalytics_MusarathR/Twitter_Analytics_App_Code/Templates/'+hashtag+'_report.html','w')
    f.write(html_string)
    f.close()
    
    return

## TWITTER ANALYTICS APPLICATION

In [35]:
global str
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

# to get information from my-form page
@app.route('/')
def my_form():
    return render_template('my-report.html')

# to post the result to web page
@app.route('/',methods=['POST'])
def my_hashtag_report():

    # read the batch file
    hashtag = str(request.form['string'])   # get the url and store in varaible
    print(hashtag)

    # get data
    tweets_data = twitter_data_download(hashtag)

    # tag the tweets
    df_tweets = twitter_abstract(tweets_data,xtrain_tfidf_word,train_y)
    # sentiment
    df_tweets = tweets_sentiment(df_tweets)
    # polarity
    df_tweets = tweets_polarity(df_tweets)
    # sentiment categorize
    df_tweets = sentiment_categorize(df_tweets)

    # plot generations
    tweets_over_time_plot_url = tweets_over_time(df_tweets)
    tweets_sentiment_plot_url = sentiment_analysis_plot(df_tweets,hashtag)
    all_tweets_table = tweets_table(df_tweets,hashtag)
    sentiment_spiral_plot_url = tweets_spiral(df_tweets)
    # create report
    html_string = define_html_string(hashtag,tweets_over_time_plot_url,tweets_sentiment_plot_url,all_tweets_table,sentiment_spiral_plot_url)
    write_to_html(hashtag,html_string)

    # open the report
    #webbrowser.open_new_tab( 'C:/Users/Musarath/Musarath/Anly699_Project/'+hashtag+'_report.html')    

    filename = str(hashtag+'_report.html')
    return render_template(filename)
        
        
# main function
if __name__ == '__main__':
    app.run()

TypeError: new_run() got an unexpected keyword argument 'host'